In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import model_from_json

# Define custom_objects if any custom layers or functions are used
custom_objects = {'DTypePolicy': tf.keras.mixed_precision.Policy}

# Load the model architecture from JSON
def load_model():
    try:
        with open("signlanguagedetectionmodel128x128.json", "r") as json_file:
            model_json = json_file.read()
        
        # Optional: Fix batch shape issues (only if necessary)
        model_json = model_json.replace('"batch_shape"', '"batch_input_shape"')

        model = model_from_json(model_json, custom_objects=custom_objects)
        model.load_weights("signlanguagedetectionmodel128x128.h5")
        print("Model loaded successfully!")
        return model
    except Exception as e:
        print(f"Error loading the model: {e}")
        exit()

# Preprocess the input image
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    image = cv2.resize(image, (128, 128))  # Resize
    image = image.astype(np.float32) / 255.0  # Normalize
    return image.reshape(1, 128, 128, 1)  # Reshape for model input

# Initialize the webcam
def initialize_camera():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Unable to access the webcam.")
        exit()
    return cap

# Main function to run the detection
def main():
    model = load_model()
    cap = initialize_camera()

    labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ["Blank"]
    print("Press 'ESC' to exit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from webcam.")
            break

        # Define region of interest (ROI)
        cv2.rectangle(frame, (0, 40), (300, 300), (0, 165, 255), 1)
        roi = frame[40:300, 0:300]

        # Preprocess and make prediction
        processed_image = preprocess_image(roi)
        try:
            pred = model.predict(processed_image, verbose=0)
            predicted_label = labels[np.argmax(pred)]
            accuracy = f"{np.max(pred) * 100:.2f}%"
        except Exception as e:
            print(f"Prediction error: {e}")
            predicted_label, accuracy = "Error", "0.00%"

        # Display the prediction
        cv2.rectangle(frame, (0, 0), (300, 40), (0, 165, 255), -1)
        cv2.putText(frame, f"{predicted_label} {accuracy}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow("Sign Language Detection", frame)

        # Break the loop if 'ESC' key is pressed
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Model loaded successfully!
Press 'ESC' to exit.
